<a href="https://colab.research.google.com/github/zihvvan/DeepLearning/blob/main/zihvvan/ch09_DL_11_RNN_TEXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 영화 리뷰 감정 분석
* RNN은 동영상, 자연어, 주가 등 동적인 데이터를 이용할 때 성능이 극대화
* 자연어 : 일상적으로 사용하는 말을 프로그래밍 언어와 구분하여 부르는 말

> 자연어에서 감정 분석과 감성 분석이 다른 말이야?
* https://shareg.pt/IZAnp6f


* IMDB 데이터셋
    * 텍스트 형태의 데이터셋
    * 50,000건의 영화 리뷰로 이루어져 있음
    * 각 리뷰는 다수의 영어 문장들로 이루어져 있으며, 평점이 7점 이상의 긍정적인 영화 리뷰는 2로, 평점이 4점 이하인 부정적인 영화 리뷰는 1로 레이블링

* 영화 리뷰 텍스트를 RNN에 입력시켜 영화평의 전체 내용을 압축하고, 이렇게 압축된 리뷰 긍정적인지 부정적인지 판단해주는 간단한 분류 모델을 생성
* 한글 데이터도 적용

* 워드 임베딩 word embedding: 언어의 최소 단위 -> 토큰화 => 벡터화 (라벨 인코딩)

## 자연어 전처리

In [1]:
!pip show torch torchtext
# torch 1.9.1 / torchtext 0.10.1 (0.13.0 이전 사용)

Name: torch
Version: 1.9.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /root/.local/lib/python3.9/site-packages
Requires: typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision, triton
---
Name: torchtext
Version: 0.10.1
Summary: Text utilities and datasets for PyTorch
Home-page: https://github.com/pytorch/text
Author: PyTorch core devs and James Bradbury
Author-email: jekbradbury@gmail.com
License: BSD
Location: /root/.local/lib/python3.9/site-packages
Requires: numpy, requests, torch, tqdm
Required-by: 


In [2]:
# 특수한 버전으로 만들어주겠다 -> 런타임 다시 시작
!pip install torchtext==0.10.1 --user -q

In [3]:
!pip show torch torchtext

Name: torch
Version: 1.9.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /root/.local/lib/python3.9/site-packages
Requires: typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision, triton
---
Name: torchtext
Version: 0.10.1
Summary: Text utilities and datasets for PyTorch
Home-page: https://github.com/pytorch/text
Author: PyTorch core devs and James Bradbury
Author-email: jekbradbury@gmail.com
License: BSD
Location: /root/.local/lib/python3.9/site-packages
Requires: numpy, requests, torch, tqdm
Required-by: 


In [4]:
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot
!pip install cartopy
import cartopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
E: Package 'libfluidsynth1' has no installation candidate
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# 필수 라이브러리 임포트
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets

In [6]:
# 하이퍼 패러미터 정의
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(f'running in {DEVICE}')

running in cuda


In [7]:
# 데이터 로딩하기
TEXT = data.Field(sequential=True, batch_first=True, lower=True) # 데이터를 받아줄 빈 객체
# sequential : 순서가 있는 데이터인가? / batch_first : 배치 사이즈 차원을 맨 앞으로 두겠다 / lower : 모두 소문자로 전처리
LABEL = data.Field(sequential=False, batch_first=True) # 데이터를 받아줄 빈 객체
# 0, 1 긍부정 여부 데이터

In [8]:
# 훈련셋, 테스트셋으로 분리
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:08<00:00, 10.2MB/s]


In [9]:
print(f"훈련셋 : {len(trainset)}")
print(f"시험셋 : {len(testset)}")

훈련셋 : 25000
시험셋 : 25000


In [10]:
trainset[0].text[:10] # 자동으로 전처리가 되어 토큰화된 데이터 묶음

['the',
 'documentary',
 'presents',
 'an',
 'original',
 'theory',
 'about',
 '"guns,',
 'germs',
 'and']

In [11]:
# 단어 사전 (vocab : 어휘)
TEXT.build_vocab(trainset, min_freq=5) # 5번 이상 등장한 단어로 단어 사전 만들기
# 학습데이터에서 5번 미만 등장한 데이터는 unk(unknown)으로 대체
LABEL.build_vocab(trainset)

In [12]:
TEXT.vocab.freqs.most_common(100) # 단어의 등장 빈도

[('the', 322198),
 ('a', 159953),
 ('and', 158572),
 ('of', 144462),
 ('to', 133967),
 ('is', 104171),
 ('in', 90527),
 ('i', 70480),
 ('this', 69714),
 ('that', 66292),
 ('it', 65505),
 ('/><br', 50935),
 ('was', 47024),
 ('as', 45102),
 ('for', 42843),
 ('with', 42729),
 ('but', 39764),
 ('on', 31619),
 ('movie', 30887),
 ('his', 29059),
 ('are', 28743),
 ('not', 28597),
 ('film', 27777),
 ('you', 27564),
 ('have', 27344),
 ('he', 26177),
 ('be', 25691),
 ('at', 22731),
 ('one', 22480),
 ('by', 21976),
 ('an', 21240),
 ('they', 20624),
 ('from', 19934),
 ('all', 19740),
 ('who', 19407),
 ('like', 18779),
 ('so', 18099),
 ('just', 17309),
 ('or', 16769),
 ('has', 16570),
 ('her', 16540),
 ('about', 16486),
 ("it's", 15970),
 ('some', 15280),
 ('if', 15189),
 ('out', 14510),
 ('what', 14055),
 ('very', 13633),
 ('when', 13609),
 ('more', 13170),
 ('there', 13094),
 ('she', 12234),
 ('would', 12027),
 ('even', 12010),
 ('good', 11926),
 ('my', 11766),
 ('only', 11566),
 ('their', 11317)

In [13]:
LABEL.vocab.freqs # pos : positive 긍정적인, nes : negative 부정적인

Counter({'pos': 12500, 'neg': 12500})

In [14]:
# 학습용 데이터를 학습셋 80% 검증셋 20%로 나누기 (validation)
trainset, valset = trainset.split(split_ratio=0.8) # 0.8 : 학습셋, (1-0.8=0.2) 검증셋

In [15]:
BATCH_SIZE

64

In [16]:
# 배치 로딩을 위한 데이터 로더
# 텍스트 형태의 데이터도 모든 학습 데이터를 한 번에 처리 X
# batch(배치) 단위로 쪼개서 학습을 진행 -> 반복할 때마다 배치를 생성해주는 반복자(iterator)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (trainset, valset, testset),
    batch_size=BATCH_SIZE,
    shuffle=True, repeat=False
)
# 반복자 -> enumerate() 함수에 넣어서 반복해주면(루프) 배치 단위의 데이터셋만 뽑아올 수 있음

In [17]:
# 사전 속 단어의 개수, 레이블의 수
vocab_size = len(TEXT.vocab)
n_classes = 2

In [18]:
print(f"[학습셋] {len(trainset)} / [검증셋] {len(valset)} / 테스트셋 {len(testset)} / [단어수] {vocab_size} / [클래스(라벨)] {n_classes}")

[학습셋] 20000 / [검증셋] 5000 / 테스트셋 25000 / [단어수] 46159 / [클래스(라벨)] 2


## RNN 모델 구현

In [19]:
class BasicGRU(nn.Module):
    def __init__(self,
                 n_layers, # 층 -> 은닉 벡터들의 층의 갯수 : 엄청 복잡한 모델이 아니라면, n_layers는 2 이하 통상 정의
                 hidden_dim, # 층 내부의 너비
                 n_vocab, # 단어 임베딩 관련
                 embed_dim, # 단어 임베딩 관련
                 n_classes, # 최종적으로 분류할 텍스트의 가짓수 (neg, pos)
                 dropout_p=0.2
                 ):
        super(BasicGRU, self).__init__()
        self.n_layers = n_layers
        # 단어 임베딩 -> 단어들을 벡터화(라벨링)
        self.embed = nn.Embedding(n_vocab, embed_dim)
        # n_vocab : 전체 데이터셋의 모든 단어를 사전 형태로 나타냈을 때, 그 사전에 등재된 단어 수
        # embed_dim : 임베딩 -> 단어 텐서가 가지는 차원값
        # RNN을 통해 생성되는 은닉 벡터의 차원값과 드롭아웃을 정의
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)

        # 단어 임베딩을 거친 텐서가 GRU로 입력
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers, batch_first=True)
        # 시계열(순서) 데이터 -> 하나의 텐서 압축
        self.out = nn.Linear(hidden_dim, n_classes)
    
    def forward(self, x): # 문장들 
        x = self.embed(x) # 단어들이 일괄적으로 숫자값을 가지는 텐서로 변환
        h_0 = self._init_state(batch_size=x.size(0)) # 젓번째 은닉 벡터 초기값 (H0)
        x, _ = self.gru(x, h_0) # 입력 x를 첫번째 은닉 벡터 h_0과 함께 gru에 입력하면
        # -> 은닉 벡터들이 시계열 배열 형태로 반환 -> (batch_size, 입력 x의 길이, hidden_dim) 3차원 텐서
        # gru를 통해 나온 텐서 x -> [:, -1, :]
        # -> 배치 내 모든 시계열 은닉 벡터들의 마지막 토큰들을 내포한 (batch_size, 1, hidden_dim)의 텐서를 추출
        # => 텐서 내에 가장 최신 은닉값
        h_t = x[:, -1, :] # 모든 데이터를 반영한 핵심값 (압축된 은닉 벡터)
        # 특정한 영화평, 영화리뷰에 있는 모든 단어(토큰)들을 압축한 값
        h_t = self.dropout(h_t)
        logit = self.out(h_t)
        return logit

    def _init_state(self, batch_size=1):
        # parameters() : 해당 신경망 모듈(nn.Module)의 가중치 정보를 iterator 형태로 반환
        weight = next(self.parameters()).data # 실제 모델의 가중치
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        # 기존에 존재하던 가중치와 연결한 뒤 순전파에 사용되는 모양으로 바꾸고, 0으로 초기화

In [20]:
# 학습함수
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        # b : 각 행의 인덱스, batch : 미니 배치 (학습을 위해 구분한 배치 사이즈 데이터)
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        # y = pos, neg -> vocab -> 1, 2 -> 0, 1
        y.data.sub_(1) # y값에 일괄적으로 1를 빼줘서, 1과 2를 0과 1로 변환
        optimizer.zero_grad() # 최적화함수 -> 기울기 계산 리셋
        # x를 모델에 입력해서 예측값인 logit 계산
        logit = model(x) # logit = 0과 1 사이의 확률
        # 손실 함수 -> 손실
        loss = F.cross_entropy(logit, y) # 예측값과 정답값(라벨) 비교 -> 손실함수
        loss.backward() # 역전파
        optimizer.step() # 가중치 갱신

In [21]:
# 평가함수 (성능 측정을 위한)
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        # 기울기 계산 안함 (optimizer 있을 필요 X)
        # x를 모델에 입력해서 예측값인 logit 계산
        logit = model(x) # logit = 0과 1 사이의 확률
        # 손실 함수 -> 손실
        loss = F.cross_entropy(logit, y, reduction='sum') # 오차의 합
        total_loss += loss.item()
        # 예측값과 정답값이 일치하는 경우
        # tensor.max(0) = 각 열에서의 최대값
        # tensor.max(1) = 각 행에서의 최대값 / [1] -> 최대값의 인덱스 (0,1)
        # max의 결과값 -> (최대값[0], 그 값의 인덱스(indices)[1])
        # view -> y와 비교할 수 있게 같은 모양 변형 => 일치하는 건 True(1) 일치하지 않는 건 False(0)
        # sum -> 일치하는 것들의 갯수만.
        # https://velog.io/@jarvis_geun/torch.argmax-torch.max
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset) # 전체 데이터의 갯수
    avg_loss = total_loss / size # 오차 평균
    avg_accuracy = corrects / size * 100 # (%) - 정확도 평균
    return avg_loss, avg_accuracy

In [22]:
# 모델 객체 정의
# 1 : 내부 GRU의 층 개수
# 256 : 모델 내 은닉 벡터의 차원값
# 128 : 임베딩(벡터화)된 토큰의 차원값
# 0.5 : 드롭아웃 비중
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)

In [23]:
# 최적화함수
# 최적화함수를 뭘 쓸지 모르면 Adam 써라!
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [24]:
# 학습
best_val_loss = None # 검증 오차를 최소화
for epoch in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    # print("[에포크: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" % (epoch, val_loss, val_accuracy))
    # https://ooyoung.tistory.com/87
    print(f"[에포크: {epoch}] 검증 오차:{val_loss:5.2f} | 검증 정확도:{val_accuracy:5.2f}")

    # 검증 오차가 가장 적은 최적의 모델
    if not best_val_loss or val_loss < best_val_loss:
        # not best_val_loss = None은 not으로 취급 = 아직 최적 검증 오차가 비어있거나
        # val_loss < best_val_loss = 새롭게 구한 검증 오차가 기존에 최적 검증 오차보다 작으면
        # => 새로운 모델의 오차가 더 작으면
        if not os.path.isdir("snapshot"): # 스냅샷 폴더 없으면 만들어주세요
            os.makedirs("snapshot") # snapshot 폴더에 저장할 것임
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss # 오차 갱신

[에포크: 1] 검증 오차: 0.70 | 검증 정확도:50.52
[에포크: 2] 검증 오차: 0.69 | 검증 정확도:51.02
[에포크: 3] 검증 오차: 0.69 | 검증 정확도:51.56
[에포크: 4] 검증 오차: 0.63 | 검증 정확도:65.30
[에포크: 5] 검증 오차: 0.43 | 검증 정확도:80.80
[에포크: 6] 검증 오차: 0.34 | 검증 정확도:86.02
[에포크: 7] 검증 오차: 0.35 | 검증 정확도:86.46
[에포크: 8] 검증 오차: 0.36 | 검증 정확도:86.22
[에포크: 9] 검증 오차: 0.40 | 검증 정확도:85.70
[에포크: 10] 검증 오차: 0.45 | 검증 정확도:85.38


In [25]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.36 | 테스트 정확도: 84.88


# 한글 : 네이버 영화 리뷰 감정 분석

In [26]:
# https://velog.io/@heiswicked/M1-Part11-%EB%B3%B5%EB%B6%88%EB%B3%B5%EC%84%A4%EC%B9%98-konlpy.tag-MECAB-on-M1-ver.221230
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 45.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.5/581.5 KB 38.0 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1251 sha256=782872f07fa1174057486bce92396747ea394099c6b79488439d00e8c9776259
  Stored in directory: /root/.cache/pip/wheels/63/09/15/cc401a7f8d041043978f3f60e64f7d65014522e104b7c9d1f2
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0     

In [27]:
# https://konlpy.org/ko/latest/index.html
from konlpy.tag import Mecab
import pandas as pd

# https://github.com/e9t/nsmc/
data_path = 'https://raw.githubusercontent.com/e9t/nsmc/master/'

In [28]:
tokenizer = Mecab() # 형태소 분석기를 통한 토큰화

# 훈련 데이터셋 & 시험 데이터셋
trainset = pd.read_csv(data_path + "ratings_train.txt", sep='\t') # tab으로 구분되어 있기 때문에 \t를 구분자로 해서 read
testset = pd.read_csv(data_path + "ratings_test.txt", sep='\t')

In [29]:
trainset.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [30]:
trainset.drop(columns=['id'], inplace=True)
testset.drop(columns=['id'], inplace=True)

In [31]:
trainset.head()

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [32]:
trainset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  149995 non-null  object
 1   label     150000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [33]:
from sklearn.model_selection import train_test_split
# 1. 결측치 (nan) 제거
# 2. 훈련셋에서 30% -> 검증셋으로 분리
train_data = trainset.dropna() # 말뭉치에서 nan 값을 제거
test_data = testset.dropna()
train_data, valid_data = train_test_split(train_data, test_size=0.3, random_state=71)

In [34]:
# TEXT, Label
TEXT = data.Field(sequential=True, use_vocab=True, tokenize=tokenizer.morphs,
                  lower=False, batch_first=True, fix_length=20) # 모든 품사 (모든 단어)
# TEXT = data.Field(sequential=True, use_vocab=True, tokenize=tokenizer.nouns,
#                   lower=False, batch_first=True) # 명사만
# LABEL = data.LabelField(batch_first=True, dtype=torch.float)
LABEL = data.Field(sequential=False, batch_first=True)

In [35]:
# 파이토치용 (텍스트) 데이터셋으로 변환
# input_data = pandas df => iterrows() => row (document, label) => TEXT, LABEL
def convert_dataset(input_data, text, label):
    list_of_example = [data.Example.fromlist(row.tolist(),
                                             fields=[('text', text), ('label', label)])
                        for _, row in input_data.iterrows()]
    dataset = data.Dataset(examples=list_of_example, fields=[('text', text), ('label', label)])
    return dataset

In [36]:
train_data = convert_dataset(train_data, TEXT, LABEL)
valid_data = convert_dataset(valid_data, TEXT, LABEL)
test_data = convert_dataset(test_data, TEXT, LABEL)

In [37]:
train_data.examples[0].text, train_data.examples[0].label

(['라이터', '던지', '는', '신', '에서', '개', '뿜', '었', '다'], 0)

In [38]:
train_data.examples[0].text, train_data.examples[0].label

(['라이터', '던지', '는', '신', '에서', '개', '뿜', '었', '다'], 0)

In [39]:
train_data.examples[10].text, train_data.examples[10].label

(['중국', '영화', 'ㄷ', 'ㄷ', 'ㄷ', 'ㄷ', 'ㄷ', 'ㄷ', '액션', '은', '역시', '헐리우드', '급'], 1)

In [40]:
# 단어 사전
TEXT.build_vocab(train_data, max_size=10000)
# TEXT.build_vocab(train_data, min_freq=5)
LABEL.build_vocab(train_data)

In [41]:
TEXT.vocab.freqs.most_common(10)

[('.', 112034),
 ('이', 51685),
 ('는', 46704),
 ('영화', 40511),
 ('다', 38802),
 ('고', 32554),
 ('하', 31100),
 ('의', 24024),
 ('도', 23881),
 ('가', 23385)]

In [42]:
# BATCH_SIZE = 64
# 학습에 사용될 iter 정의
train_iter, val_iter, test_iter = data.Iterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    shuffle=True, repeat=False, sort=False, device=DEVICE
)

In [43]:
# vocab_size
vocab_size = len(TEXT.vocab)

In [44]:
vocab_size

10002

In [45]:
# n_classes = 2
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
# model = BasicGRU(2, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)

In [46]:
lr = 0.001
# lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [47]:
# 학습
EPOCHS = 10
# EPOCHS = 20
best_val_loss = None # 검증 오차를 최소화
for epoch in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print(f"[에포크: {epoch}] 검증 오차:{val_loss:5.2f} | 검증 정확도:{val_accuracy:5.2f}")

    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"): 
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification_ko.pt')
        best_val_loss = val_loss

[에포크: 1] 검증 오차: 0.37 | 검증 정확도:83.13
[에포크: 2] 검증 오차: 0.35 | 검증 정확도:84.38
[에포크: 3] 검증 오차: 0.35 | 검증 정확도:84.74
[에포크: 4] 검증 오차: 0.38 | 검증 정확도:84.65
[에포크: 5] 검증 오차: 0.41 | 검증 정확도:84.36
[에포크: 6] 검증 오차: 0.48 | 검증 정확도:83.87
[에포크: 7] 검증 오차: 0.54 | 검증 정확도:84.07
[에포크: 8] 검증 오차: 0.62 | 검증 정확도:83.68
[에포크: 9] 검증 오차: 0.63 | 검증 정확도:83.65
[에포크: 10] 검증 오차: 0.66 | 검증 정확도:83.64


In [48]:
model.load_state_dict(torch.load('./snapshot/txtclassification_ko.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.35 | 테스트 정확도: 84.25
